In [53]:
import boto3
import zipfile
import os

In [54]:
AWSsession = boto3.Session(
    aws_access_key_id='YOUR_AWS_ACCESS_KEY',
    aws_secret_access_key='YOUR_AWS_SECRET_ACCESS_KEY',
    region_name='eu-north-1'
)
s3_client = AWSsession.client('s3')

In [55]:
bucket_name = "my-flight-data"

create_bucket_config = {
    'LocationConstraint': 'eu-north-1'
}

s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=create_bucket_config)

print("Bucket created successfully.")


Bucket created successfully.


In [56]:
def zip_file(source_file, destination_file):
    with zipfile.ZipFile(destination_file, 'w') as zipf:
        zipf.write(source_file, os.path.basename(source_file))

In [57]:
csv_file_path = "flight_data.csv"


In [58]:
zipped_file_name = "flight_data.zip"

In [59]:
zip_file(csv_file_path, zipped_file_name)

In [60]:
s3_client.upload_file(zipped_file_name, bucket_name, zipped_file_name)
#os.remove(zipped_file_name)
print("CSV file zipped and uploaded to S3 successfully.")


CSV file zipped and uploaded to S3 successfully.


In [ ]:
# COPY FILE FROM S3 TO REDSHIFT

In [61]:
import psycopg2
import logging

In [62]:
def extract_csv_from_zip(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('flight_data')

In [63]:
def upload_csv_to_s3(csv_path):
    s3 = boto3.client('s3', aws_access_key_id="YOUR_AWS_ACCESS_KEY",
                       aws_secret_access_key="YOUR_AWS_SECRET_ACCESS_KEY",
                         region_name="eu-north-1"
                    )
    
    with open(csv_path, 'rb') as data:
        s3.upload_fileobj(data, bucket_name, csv_file_path)

In [64]:
extract_csv_from_zip('flight_data.zip')

In [65]:
upload_csv_to_s3('flight_data/flight_data.csv')

In [66]:
try:
    con = psycopg2.connect(
        dbname = "YOUR_REDSHIFT_DB_NAME",
        host = "YOUR_REDSHIFT_HOST_NAME",
        port = "5439",
        user = "YOUR_REDSHIFT_USER_NAME",
        password = "YOUR_REDSHIFT_PASSWORD"
    )
    logging.info('Redshift connection successful')
except Exception as e:
    logging.exception(e)    

In [67]:
con.autocommit = True

In [68]:
cur = con.cursor()

In [69]:
create_schema_query = """
CREATE SCHEMA IF NOT EXISTS flight_schema;
"""
cur.execute(create_schema_query)
con.commit()

In [70]:
create_table_query = """
CREATE TABLE IF NOT EXISTS flight_schema.flight (
    flight_date VARCHAR(255),
    flight_status VARCHAR(255),
    aircraft VARCHAR(255),
    live VARCHAR(255),
    departure_airport VARCHAR(255),
    departure_timezone VARCHAR(255),
    departure_iata VARCHAR(255),
    departure_icao VARCHAR(255),
    departure_terminal VARCHAR(255),
    departure_gate VARCHAR(255),
    departure_delay VARCHAR(255),
    departure_scheduled VARCHAR(255),
    departure_estimated VARCHAR(255),
    departure_actual VARCHAR(255),
    departure_estimated_runway VARCHAR(255),
    departure_actual_runway VARCHAR(255),
    arrival_airport VARCHAR(255),
    arrival_timezone VARCHAR(255),
    arrival_iata VARCHAR(255),
    arrival_icao VARCHAR(255),
    arrival_terminal VARCHAR(255),
    arrival_gate VARCHAR(255),
    arrival_baggage VARCHAR(255),
    arrival_delay VARCHAR(255),
    arrival_scheduled VARCHAR(255),
    arrival_estimated VARCHAR(255),
    arrival_actual VARCHAR(255),
    arrival_estimated_runway VARCHAR(255),
    arrival_actual_runway VARCHAR(255),
    airline_name VARCHAR(255),
    airline_iata VARCHAR(255),
    airline_icao VARCHAR(255),
    flight_number VARCHAR(255),
    flight_iata VARCHAR(255),
    flight_icao VARCHAR(255),
    codeshared_airline_name VARCHAR(255),
    codeshared_airline_iata VARCHAR(255),
    codeshared_airline_icao VARCHAR(255),
    codeshared_flight_number VARCHAR(255),
    codeshared_flight_iata VARCHAR(255),
    codeshared_flight_icao VARCHAR(255)
);
"""
cur.execute(create_table_query)
con.commit()

In [72]:
copy_command = """
COPY flight_schema.flight
FROM 's3://my-flight-data/flight_data.csv'
IAM_ROLE 'YOUR_ARN'
IGNOREHEADER 1
DELIMITER ','
CSV
REGION 'eu-north-1';
"""
cur.execute(copy_command)
con.commit()

In [73]:
cur.close()
con.close()

print("Data loaded into Redshift successfully.")

Data loaded into Redshift successfully.
